# Sujet 2: Modélisation des interdépendances

**Quel est l’impact d’une baisse de la capacité de LNG sur le système en général et en particulier sur
le système électrique ?**

Pour aller plus loin : modéliser le stockage gaz

**Authors**: Lucas DEFRANCE, Quentin DEHAENE, Jules PAINDESSOUS--POETE

## Imports

In [ ]:
# Necessary imports
using JuMP
using XLSX
using DataFrames
#use the solver you want
using HiGHS